In [1]:
%matplotlib inline

import os
import shapely
import earthaccess
import numpy as np
import rasterio as rio
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [4]:
# Load the search area polygon (replace 'search_area.shp' with your actual shapefile)
AOI_path = 'Data_files/Area_of_interest.shp'
AOI = gpd.read_file(AOI_path)

DriverError: Assignment/Data_files/Area_of_interest.shp: No such file or directory

In [ ]:
# Check if the CRS is a projected coordinate system (not WGS84)
if AOI_gdf.crs != 'epsg:4326':
    print("Attention: The coordinate system of 'search_area' must be in geographic coordinates (WGS84, EPSG:4326). "
          "Please convert the CRS to WGS84 before proceeding with earthaccess data search to prevent runtime errors.")
else:
    print("The CRS of 'search_area' is correctly set to WGS84 (EPSG:4326). Proceeding with earthaccess data search.")

In [ ]:
# You need to set the CRS for 'AOI_gdf' before transforming it
AOI.crs = {'init': 'epsg:32633'}

# Transform the CRS to WGS84 (EPSG:4326)
AOI_wgs84 = AOI.to_crs('+proj=longlat +datum=WGS84')

In [ ]:
AOI_wgs84.crs

In [ ]:
earthaccess.login(strategy='netrc')

outline = AOI_wgs84['geometry'].unary_union

# get the min x, min y, max x, max y values of the Area of Interest
outline.bounds

search_area = shapely.geometry.polygon.orient(outline, sign=1) # a sign of 1 means oriented counter-clockwise

search_area # this doesn't actually change the geometry, just the order of the vertices

In [ ]:
from earthaccess import DataCollections
import datetime

# Define the time range for the data search
start_date = datetime.datetime(2022, 1, 1)
end_date = datetime.datetime(2022, 12, 31)

In [ ]:
datasets = earthaccess.search_datasets(
    keyword="Sentinel-2", # search for datasets that match the keyword "Sentinel-2"
    polygon=search_area.exterior.coords, # search for datasets that intersect AOI,
    temporal=(start_date, end_date)
)

In [ ]:
dataset = datasets[0] # get the first result
dataset.get_umm('EntryTitle') # fill this in with the metadata field that you want

In [ ]:
ds_name = dataset.get_umm('ShortName') # fill in the following with the correct field name to return the short name of the dataset

print(f"Dataset short name: {ds_name}")

In [ ]:
results = earthaccess.search_data(
    short_name=ds_name, # search for Sentinel-2 granules
    polygon=search_area.exterior.coords, # search for images that intersect our AOI
    count=10, # only show the first 10 results
    temporal=(start_date, end_date)
)

In [ ]:
granule = results[1]  # Indexing starts at 0, so 1 is the second item
granule

In [ ]:
# Define the path to the Data_files directory
data_files_directory = "Data_files"

# Create the full path for the new directory inside Data_files
new_directory_path = os.path.join(data_files_directory, ds_name)

# Create the new folder inside Data_files
os.makedirs(new_directory_path, exist_ok=True)

# Download the second granule into the new directory
downloaded_file = earthaccess.download([granule], new_directory_path)

In [ ]:
# Open the raster file and check its CRS
src = rasterio.open('Data_files/HLSS30/HLS.S30.T33TXF.2022004T094309.v2.0.B04.tif')

src.crs

In [ ]:
from rasterio.mask import mask

# Open the red and NIR bands
with rasterio.open('Data_files/HLSS30/HLS.S30.T33TXF.2022004T094309.v2.0.B04.tif') as red_src, \
     rasterio.open('Data_files/HLSS30/HLS.S30.T33TXF.2022004T094309.v2.0.B08.tif') as nir_src:

    # Read the raster data for the bands
    red = red_src.read(1)
    nir = nir_src.read(1)

    # Calculate NDVI
    ndvi = (nir - red) / (nir + red)

    # Plot NDVI
    plt.imshow(ndvi, cmap='RdYlGn', vmin=-1, vmax=1)
    plt.colorbar()
    plt.title('NDVI Image')
    plt.show()

In [ ]:
# Define the output path for the NDVI raster file
output_ndvi_path = 'Data_files/NDVI_output.tif'

# Create a new raster file for the NDVI data
with rasterio.open(output_ndvi_path, 'w', driver='GTiff', height=ndvi.shape[0], width=ndvi.shape[1],
                       count=1, dtype=ndvi.dtype, crs=red_src.crs, transform=red_src.transform) as dst:
    dst.write(ndvi, 1)

print(f"NDVI data saved to {output_ndvi_path}")

In [ ]:
# Load the NDVI raster (replace 'Data_files/NDVI_output.tif' with your actual NDVI raster file)
ndvi_path = 'Data_files/NDVI_output.tif'
with rio.open(ndvi_path) as ndvi_src:
    ndvi_data = ndvi_src.read(1)
    ndvi_transform = ndvi_src.transform

    # Load the search area polygon (replace 'search_area.shp' with your actual shapefile)
    #AOI_path = 'Data_files/Area_of_interest.shp'
    #AOI = gpd.read_file(AOI_path)

    # Clip the NDVI data to the search area
    clipped_ndvi_data, clipped_ndvi_transform = mask(ndvi_src, shapes=AOI.geometry, crop=True)

    # Define the output path for the clipped NDVI raster file
    clipped_ndvi_path = 'Data_files/clipped_ndvi.tif'
    with rasterio.open(clipped_ndvi_path, 'w', **ndvi_src.meta) as clipped_ndvi_dst:
        clipped_ndvi_dst.write(clipped_ndvi_data)

    print(f"Clipped NDVI saved to {clipped_ndvi_path}")

In [ ]:
# Plot the clipped NDVI
plt.imshow(clipped_ndvi_data[0], cmap='RdYlGn', vmin=-1, vmax=1)
plt.colorbar()
plt.title('Clipped NDVI Image')
plt.show()

In [ ]:
# Define the directory where the Sentinel files are located
directory = "C:\\Users\\M533\\Assignment\\HLSS30"

# Function to delete files in a directory
def delete_sentinel_files(directory):
    try:
        # List all files in the directory
        files = os.listdir(directory)
        # Iterate over each file
        for file in files:
            # Construct full file path
            file_path = os.path.join(directory, file)
            # Check if it's a Sentinel file based on the extension
            if file_path.endswith('.tif'):
                # Delete the file
                os.remove(file_path)
                print(f"Deleted: {file_path}")
    except Exception as e:
        # Print the error message
        print(f"Deletion failed: {e}")

# Call the function to delete the Sentinel files
delete_sentinel_files(directory)